In [ ]:
###OBS: Deletar imports que não serão utilizados no código
import calendar
import datetime
import json
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
from multiprocessing.pool import ThreadPool
import warnings


class ANA:
    """
    It provides a connection with the Brazilian National Water Agency (Agência Nacional de Águas - ANA) database
    """
    
 @staticmethod
    def __data_ana(list_station, data_type, only_consisted, threads=10):
    if type(list_station) is not list:
        list_station = [list_station]
    data_types = {'3': ['Vazao{:02}'], '2': ['Chuva{:02}'], '1': ['Cota{:02}']}

    def __call_request(station):
        params = {'codEstacao': str(station), 'dataInicio': '', 'dataFim': '', 'tipoDados': data_type, 'nivelConsistencia': ''}
        try:
            response = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica', params,
                                    timeout=120.0)
        except (
                requests.ConnectTimeout, requests.HTTPError, requests.ReadTimeout, requests.Timeout,
                requests.ConnectionError):
            return pd.DataFrame()
        except http.client.IncompleteRead:
            try:
                response = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica',
                                        params,
                                        timeout=120.0)
            except:
                print('It was not possible to get the station {} data'.format(station))
                return pd.DataFrame()
        except:
            print('It was not possible to get the station {} data'.format(station))
            return pd.DataFrame()
        try:
            tree = ET.ElementTree(ET.fromstring(response.content))
            root = tree.getroot()
        except:
            return pd.DataFrame()

        df = []
        for month in root.iter('SerieHistorica'):
            code = month.find('EstacaoCodigo').text
            code = f'{int(code):08}'
            consist = int(month.find('NivelConsistencia').text)
            date = pd.to_datetime(month.find('DataHora').text, dayfirst=False)
            date = pd.Timestamp(date.year, date.month, 1, 0)
            last_day = calendar.monthrange(date.year, date.month)[1]
            month_dates = pd.date_range(date, periods=last_day, freq='D')
            data = []
            list_consist = []
            for i in range(last_day):
                value = data_types[params['tipoDados']][0].format(i + 1)
                try:
                    data.append(float(month.find(value).text))
                    list_consist.append(consist)
                except TypeError:
                    data.append(month.find(value).text)
                    list_consist.append(consist)
                except AttributeError:
                    data.append(None)
                    list_consist.append(consist)
            index_multi = list(zip(month_dates, list_consist))
            index_multi = pd.MultiIndex.from_tuples(index_multi, names=["Date", "Consistence"])
            df.append(pd.DataFrame({code: data}, index=index_multi))
        if (len(df)) == 0:
            return pd.DataFrame()
        df = pd.concat(df)
        df = df.sort_index()
        if not only_consisted:
            drop_index = df.reset_index(level=1, drop=True).index.duplicated(keep='last')
            df = df[~drop_index]
            df = df.reset_index(level=1, drop=True)
        else:
            df = df[df.index.get_level_values(1) == 2]
            df = df.reset_index(level=1, drop=True)
            if (len(df)) == 0:
                return pd.DataFrame()
        series = df[code]
        date_index = pd.date_range(series.index[0], series.index[-1], freq='D')
        series = series.reindex(date_index)
        return series

    if len(list_station) < threads:
        threads = len(list_station)

    with ThreadPool(threads) as pool:
        responses = list(tqdm(pool.imap(__call_request, list_station), total=len(list_station)))
    responses = [response for response in responses if not response.empty]
    data_stations = pd.concat(responses, axis=1)
    date_index = pd.date_range(data_stations.index[0], data_stations.index[-1], freq='D')
    data_stations = data_stations.reindex(date_index)
    return data_stations

    ###DÚVIDA: A função, quando chamada no notebook do Lucena retorna prec e não o DeprecationWarning. Por que?
    @staticmethod
    def prec_data(list_station, only_consisted=False):
        raise DeprecationWarning('The method name have changed. Use prec() instead of prec_data()')
    
    @staticmethod
    def prec(list_station, only_consisted=False, threads=10):
        """
        Get the precipitation station data series from a list of stations code.
        Parameters
        ----------
        list_station : list of strings
            A list of with the stations code as strings.
        only_consisted : boolean, default False
            If True, returns only the data classified as consistent by the provider.
        threads: int
            Number of parallel requisitions
        Returns
        -------
        data_stations : pandas DataFrame
            The data of each station as a column in a pandas DataFrame
        """

        data_stations = ANA.__data_ana(list_station, '2', only_consisted=only_consisted, threads=threads)

        return data_stations